In [ ]:
import torch
from transformers import AutoTokenizer, EsmModel
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
import os

# ⚠️ UWAGA: Ten notebook nie musi być uruchamiany ponownie!
# Embeddingi są już wygenerowane w data/gold/
# Uruchom tylko jeśli chcesz regenerować embeddingi (>60 minut)

if os.path.exists("../data/gold/X_train_esm2.npy") and os.path.exists("../data/gold/X_test_esm2.npy"):
    print("✅ Embeddingi już istnieją! Pomijam...")
    print("   X_train_esm2.npy")
    print("   X_test_esm2.npy")
    print("\n💡 Możesz pominąć ten notebook i przejść do 03_training_nn.ipynb")
else:
    print("⚠️ Brak embeddingów, będę je generować...")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "facebook/esm2_t6_8M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = EsmModel.from_pretrained(model_name).to(device).eval()

def generate_embeddings(fasta_path, output_npy, ids_output_npy):
    """Generuje embeddingi z error handlingiem"""
    
    # Wczytaj sekwencje
    records = []
    skipped = 0
    
    for r in SeqIO.parse(fasta_path, "fasta"):
        prot_id = r.id.split('|')[1] if '|' in r.id else r.id
        seq = str(r.seq)
        
        # 🆕 DODAJ: Filtrowanie zbyt długich sekwencji
        if len(seq) > 1024:
            seq = seq[:1024]
            skipped += 1
        
        # 🆕 DODAJ: Usuwanie nietypowych aminokwasów
        seq = ''.join([aa if aa in 'ACDEFGHIKLMNPQRSTVWY' else 'X' for aa in seq])
        
        records.append({"ID": prot_id, "Seq": seq})
    
    df = pd.DataFrame(records)
    
    print(f"📊 Statystyki:")
    print(f"  Białka: {len(df):,}")
    print(f"  Obcięte (>1024aa): {skipped}")
    print(f"  Średnia długość: {df['Seq'].str.len().mean():.1f} aa")
    
    # Generowanie embeddingów
    all_embs = []
    batch_size = 32
    
    # 🆕 DODAJ: Progress bar z dodatkową informacją
    pbar = tqdm(range(0, len(df), batch_size), desc="Embedding batches")
    
    for i in pbar:
        batch_seqs = df['Seq'].iloc[i : i + batch_size].tolist()
        
        try:
            inputs = tokenizer(
                batch_seqs, 
                return_tensors="pt", 
                padding=True, 
                truncation=True, 
                max_length=1024
            ).to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                all_embs.append(emb)
            
            # 🆕 DODAJ: Update progress bar z info
            pbar.set_postfix({
                'batch': f'{i//batch_size + 1}/{len(df)//batch_size + 1}',
                'proteins': f'{min(i + batch_size, len(df))}/{len(df)}'
            })
            
        except Exception as e:
            print(f"\n⚠️ Error w batchu {i}: {e}")
            # Spróbuj pojedynczo
            for seq in batch_seqs:
                try:
                    inputs = tokenizer([seq], return_tensors="pt", truncation=True, max_length=1024).to(device)
                    with torch.no_grad():
                        outputs = model(**inputs)
                        emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                        all_embs.append(emb)
                except:
                    # Jeśli nadal błąd, użyj zerowych embeddingów
                    all_embs.append(np.zeros((1, 320)))
    
    # Zapis
    final_embs = np.vstack(all_embs)
    np.save(output_npy, final_embs)
    np.save(ids_output_npy, df['ID'].values)
    
    print(f"\n✅ Zapisano embeddingi: {final_embs.shape}")

# Reszta bez zmian
print("🧬 Generowanie embeddingów TRAIN...")
generate_embeddings(
    "../data/bronze/Train/train_sequences.fasta", 
    "../data/gold/X_train_esm2.npy", 
    "../data/gold/train_ids.npy"
)

print("\n🧬 Generowanie embeddingów TEST...")
generate_embeddings(
    "../data/bronze/Test/testsuperset.fasta", 
    "../data/gold/X_test_esm2.npy", 
    "../data/gold/test_protein_ids.npy"
)

print("\n✅ Gotowe! Wszystkie embeddingi zapisane.")

/home/screachail/projects/bioinformatics-portfolio/projects/02-cafa6-protein-function-prediction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🧬 Generowanie embeddingów TRAIN...


100%|██████████| 2576/2576 [19:52<00:00,  2.16it/s]


🧬 Generowanie embeddingów TEST...


100%|██████████| 7010/7010 [50:38<00:00,  2.31it/s]


✅ Gotowe! Masz pliki .npy na dysku.
